In [ ]:
!pip install numpy pandas matplotlib torch torchmetrics

In [ ]:
# Minimal PyTorch CUDA availability check - fail fast if CUDA is unavailable
try:
    import torch
except Exception as e:
    raise RuntimeError(f"PyTorch is not installed or failed to import: {e}")

# Ensure CUDA is available - if not, raise an error because we rely on GPU
if not torch.cuda.is_available():
    raise RuntimeError("CUDA is not available. Please ensure an NVIDIA GPU, drivers, and CUDA are installed and that the CUDA toolkit is compatible with your PyTorch build.")

# Report devices
print('PyTorch version:', torch.__version__)
cnt = torch.cuda.device_count()
print('CUDA device count:', cnt)
for i in range(cnt):
    try:
        name = torch.cuda.get_device_name(i)
    except Exception:
        name = f'unknown-device-{i}'
    print(f'Device {i}:', name)

# Quick allocation test to confirm functional GPU access
try:
    _ = torch.zeros(1, device='cuda')
    print('Successfully allocated a tensor on CUDA.')
except Exception as e:
    raise RuntimeError('CUDA appears available but tensor allocation failed: ' + str(e))

# may need to use different dgl if not on linux or using different version of CUDA
!pip install  dgl -f https://data.dgl.ai/wheels/torch-2.1/cu121/repo.html

In [ ]:
# alternate dependencies for running on mac without CUDA (only run if previous block fails)

!pip install pydantic
!pip install PyYAML
!pip install numpy==1.26.4
!pip install torch==2.1.1 torchdata==0.7.1
!pip install dgl -f https://data.dgl.ai/wheels/repo.html

import torch
import dgl

In [ ]:
import pandas as pd
from pathlib import Path

# Path to the small transactions CSV (relative to this notebook).
DATA_PATH = Path("dataset") / "HI-Small_Trans.csv"

# Load into a DataFrame
small_trans = pd.read_csv(DATA_PATH)

# Quick summary and preview
print(f"Loaded {len(small_trans)} rows; columns: {list(small_trans.columns)}")
small_trans.head()

In [ ]:
# Basic analysis: currencies, banks, and other summaries
# This cell is robust to different column names: it searches for currency-like and bank-like columns

# Show shape and a small sample
rows, cols = small_trans.shape
print(f"Data shape: {rows} rows x {cols} columns")
print()
print("Sample rows:")
display(small_trans.head())

# Missing values by column (top 10)
missing_by_col = small_trans.isnull().sum().sort_values(ascending=False).head(15)
print("Top missing values by column:")
print(missing_by_col.to_string())
print()

# Find likely currency column(s)
currency_candidates = [c for c in small_trans.columns if any(k in c.lower() for k in ('currency','ccy','curr'))]
if currency_candidates:
    cur_col = currency_candidates[0]
    num_currencies = small_trans[cur_col].nunique(dropna=True)
    top_currencies = small_trans[cur_col].value_counts().head(10)
    print(f"Found currency column: '{cur_col}' — {num_currencies} unique values")
    print("Top currencies (by count):")
    print(top_currencies.to_string())
else:
    cur_col = None
    print("No currency-like column found.")
    print("Columns:", list(small_trans.columns))

print()
# Find likely bank-related columns
bank_candidates = [c for c in small_trans.columns if any(k in c.lower() for k in ('bank','institution','bic','iban','bankid','bank_id','bankname','bank_name'))]
if bank_candidates:
    # Count unique bank identifiers across candidate columns
    unique_banks = set()
    for c in bank_candidates:
        unique_banks.update(small_trans[c].dropna().astype(str).unique())
    num_unique_banks = len(unique_banks)
    print(f"Found bank-like columns: {bank_candidates} — approx. {num_unique_banks} unique bank identifiers (aggregated)")
else:
    num_unique_banks = None
    print("No bank-like columns found.")

print()
# Other basic summaries: amount column candidates and top senders/receivers if available
amt_candidates = [c for c in small_trans.columns if any(k in c.lower() for k in ('amount','amt','value'))]
if amt_candidates:
    amt_col = amt_candidates[0]
    print(f"Found amount column: {amt_col} — summary:")
    print(small_trans[amt_col].describe())
else:
    print("No amount-like column found.")

# If sender/receiver columns exist, show top participants
party_candidates = [c for c in small_trans.columns if any(k in c.lower() for k in ('sender','receiver','originator','beneficiary','from_','to_','account'))]
if party_candidates:
    print()
    print("Top participants in party-like columns:")
    for c in party_candidates:
        print(f"Column: {c}")
# Final small sample
display(small_trans.sample(min(5, len(small_trans))))

In [ ]:
import numpy as np

# convert hex account numbers to int
hex_to_int = np.vectorize(lambda x: int(x, 16))

# create adjacency lists to represent the graph
source = hex_to_int(small_trans['Account'])
target = hex_to_int(small_trans['Account.1'])

In [ ]:
# create directed graph using dgl
g_accounts = dgl.graph((source, target))

#remove isolated nodes
g = dgl.compact_graphs(g_accounts)

In [ ]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder

# extract individual edge features
time = pd.to_datetime(small_trans['Timestamp']).astype('int64') / 1e9
amount_paid = small_trans['Amount Paid'].to_numpy()
amount_received = small_trans['Amount Received'].to_numpy()

# use one-hot encoding for categorical variables
paid_enc = OneHotEncoder(sparse_output=False)
paid_currency = paid_enc.fit_transform(small_trans['Payment Currency'].to_numpy().reshape(-1, 1))

received_enc = OneHotEncoder(sparse_output=False)
received_currency = received_enc.fit_transform(small_trans['Receiving Currency'].to_numpy().reshape(-1, 1))

format_enc = OneHotEncoder(sparse_output=False)
pay_format = format_enc.fit_transform(small_trans['Payment Format'].to_numpy().reshape(-1, 1))

# combine edge features into single tensor
numeric_features = np.column_stack([time, amount_paid, amount_received])
edge_features = torch.cat((torch.from_numpy(numeric_features),
                           torch.from_numpy(paid_currency),
                           torch.from_numpy(received_currency),
                           torch.from_numpy(pay_format)), 1).float()

# create edge labels
fraud_label = torch.tensor(small_trans['Is Laundering'].to_numpy())

# attach features and labels to graph
g.edata['features'] = edge_features
g.edata['label'] = fraud_label

In [ ]:
# use chronological 60/20/20 train/val/test split based on documentation

num_edges = g.num_edges()
num_train = int(num_edges * 0.6)
num_val = int(num_edges * 0.8)

train_mask = torch.zeros(num_edges, dtype=torch.bool)
train_mask[:num_train] = True

val_mask = torch.zeros(num_edges, dtype=torch.bool)
val_mask[num_train:num_val] = True

test_mask = torch.zeros(num_edges, dtype=torch.bool)
test_mask[num_val:] = True

g.edata['train_mask'] = train_mask
g.edata['val_mask'] = val_mask
g.edata['test_mask'] = test_mask

In [ ]:
#use if want to create separate subgraphs for train/val/test split

#train_edges, val_edges, test_edges = dgl.data.utils.split_dataset(g.edges('eid'), frac_list=[0.6, 0.2, 0.2])

#train = dgl.edge_subgraph(g, train_edges)
#val = dgl.edge_subgraph(g, val_edges)
#test = dgl.edge_subgraph(g, test_edges)